In [1]:
# Usual
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import time
from tqdm import tqdm

# Autoreload
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
%matplotlib inline

https://docs.google.com/document/d/10sXEhzFRSnvFcl3XxNGhnD4N2SedqwdAvK3dsihxVUA/edit#

In [2]:
import pybullet

ImportError: No module named 'pybullet'

In [3]:
import pybullet_envs.bullet.minitaur_gym_env as e
env = e.MinitaurBulletEnv(render=True)

ImportError: No module named 'pybullet_envs'

# Understanding the environment

In [4]:
STATE_SIZE = env.observation_space.shape[0]
STATE_SIZE

28

In [5]:
print(env.action_space.low)
print(env.action_space.high)

[-1 -1 -1 -1 -1 -1 -1 -1]
[1 1 1 1 1 1 1 1]


In [6]:
ACTION_SIZE = env.action_space.shape[0]
ACTION_SIZE

8

# Random moves

## Try a random policy

In [7]:
np.random.random(8)*2 - 1

array([ 0.38536649,  0.56869759, -0.43308386,  0.9615505 , -0.83600312,
       -0.36766522, -0.43706896, -0.25853698])

In [8]:
def random_action():
    return np.random.random(ACTION_SIZE)*2 - 1

In [9]:
rewards = []
s_nexts = []
infos = []
s = env.reset()

for i in tqdm(range(100)):
    actions = random_action()
    env.render()
    s_next,r,done,info = env.step(actions)
    s_nexts.append(s_next)
    infos.append(info)
    rewards.append(r)

100%|████████████████████████████████████████| 100/100 [00:01<00:00, 51.89it/s]


## Explore the data returned by the environment

In [13]:
infos[0]

{}

In [14]:
rewards[:5]

[-0.00047881277044909521,
 -0.00024526470346701351,
 -0.0011944926061204396,
 -0.0011646968378199998,
 -0.0030942434154715166]

In [16]:
s_nexts[0]

array([  1.44067050e+00,   1.47115851e+00,   1.53102526e+00,
         1.52258336e+00,   1.53793137e+00,   1.54268266e+00,
         1.48200574e+00,   1.56954979e+00,  -9.38269728e+00,
        -2.77456238e+00,   2.89582127e+00,   2.37714265e+00,
         2.26310536e+00,   2.36976821e+00,  -4.29077102e+00,
         7.80139986e+00,  -1.59646062e+00,  -1.34636550e-01,
         8.59873974e-01,   2.23901040e+00,   2.74382541e+00,
         3.60712488e+00,  -8.72725117e-01,   1.96200634e+00,
        -2.23100006e-03,   1.87687846e-03,   5.42155351e-04,
         9.99995603e-01])

# First attempt with a DQN agent

In [19]:
import sys
sys.path.append("..")

In [43]:
from rl.agents.dqn_agent import DQNAgent

In [58]:
agent = DQNAgent(STATE_SIZE,ACTION_SIZE,low = -1,high = 1,action_type="continuous")

In [57]:
N_EPISODES = 1000
MAX_STEPS = 2000

In [59]:
#---------------------------------------------
# ITERATION OVER EPISODES
for i_episode in range(N_EPISODES):

    # Reset the environment
    s = env.reset()
    reward = 0
    #-----------------------------------------
    # EPISODE RUN
    for i_step in range(MAX_STEPS):

        # Render the environement
        # env.render() #and (i_step % RENDER_EVERY == 0)

        # The agent chose the action considering the given current state
        a = agent.act(s)

        # Take the action, get the reward from environment and go to the next state
        s_next,r,done,info = env.step(a)
        reward += r

        # Remember the important variables
        agent.remember(s,a,r,s_next,done)

        # Go to the next state
        s = s_next

        # If the episode is terminated
        if done or i_step == MAX_STEPS - 1:
            print("Episode {}/{} finished after {} timesteps - epsilon : {:.2} - reward : {:.2}".format(i_episode+1,N_EPISODES,i_step,agent.epsilon,reward))
            break

Episode 1/1000 finished after 1720 timesteps - epsilon : 1.0 - reward : -2.0
Episode 2/1000 finished after 158 timesteps - epsilon : 1.0 - reward : -0.06
Episode 3/1000 finished after 163 timesteps - epsilon : 1.0 - reward : -0.4
Episode 4/1000 finished after 391 timesteps - epsilon : 1.0 - reward : -0.58
Episode 5/1000 finished after 1010 timesteps - epsilon : 1.0 - reward : -1.8
Episode 6/1000 finished after 718 timesteps - epsilon : 1.0 - reward : -0.74
Episode 7/1000 finished after 220 timesteps - epsilon : 1.0 - reward : -0.34
Episode 8/1000 finished after 493 timesteps - epsilon : 1.0 - reward : -0.6
Episode 9/1000 finished after 82 timesteps - epsilon : 1.0 - reward : -0.13
Episode 10/1000 finished after 1792 timesteps - epsilon : 1.0 - reward : -2.6
Episode 11/1000 finished after 1993 timesteps - epsilon : 1.0 - reward : -3.0
Episode 12/1000 finished after 1518 timesteps - epsilon : 1.0 - reward : -2.8
Episode 13/1000 finished after 1094 timesteps - epsilon : 1.0 - reward : -1.

KeyboardInterrupt: 